In [1]:
from utils import PredictorSPlitModel, arr2db, forna_url, DataEncoder

Using TensorFlow backend.


In [2]:
# running on CPU is extremely slow!

In [3]:
import numpy as np

In [4]:
from collections import Counter

In [5]:
import keras

In [6]:
# model = Predictor('model/2019_09_08.1/model.hdf5')
model = PredictorSPlitModel('model/2019_09_12.1/model.hdf5')






4

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


utils.py:366: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  output=[layer_final_hidden.output, layer_fe.output])
utils.py:376: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ar..., inputs=[<tf.Tenso...)`
  output=new_output)


In [7]:
seq='GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU'
# y, logp, fe = model.predict_one_step_ar(seq=seq, 
#                                        n_sample=100, start_offset=2, p_clip=1e-12)

y, logp, fe = model.predict_one_step_ar(seq=seq, 
                                       n_sample=20, start_offset=2, p_clip=1e-12)


 12%|█▎        | 4/32 [00:01<00:11,  2.48it/s]utils.py:483: RuntimeWarning: divide by zero encountered in log
  _lp = vals_sampled * np.log(_vals) + (1 - vals_sampled) * np.log(1 - _vals)
100%|██████████| 32/32 [00:09<00:00,  3.74it/s]


In [8]:
y.shape

(20, 34, 34, 1)

In [9]:
db_strs = []  # collect all dot bracket notation

for idx_sample in range(y.shape[0]):  
    _pred = y[idx_sample, :, :, 0]

#     print idx_sample
#     print np.where(_pred==1)
    
#     # locate pseudo knot?
#     one_idxes = zip(np.where(_pred==1)[0], np.where(_pred==1)[1])
#     for i in range(len(one_idxes)):
#         for j in range(i+1, len(one_idxes)):
#             pair1 = one_idxes[i]
#             pair2 = one_idxes[j]
#             # see if they cross
#             if pair2[0] < pair1[1] < pair2[1]:
#                 print 'knot!'
#                 print pair1, pair2
    

    
    # dot-bracket notation
    db_str = arr2db(_pred)
    db_strs.append(db_str)
#     print idx_sample, db_str
    print idx_sample
    print forna_url(seq, db_str)
    
    print ''

[[(0, 18), (1, 17), (2, 16), (3, 15), (4, 14), (5, 12), (6, 11)]]
[0]
0
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=(((((((....)).)))))...............

[[(0, 18), (1, 17), (2, 16), (3, 15), (4, 14), (5, 12), (6, 11)]]
[0]
1
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=(((((((....)).)))))...............

[[(0, 18), (1, 17), (2, 16), (3, 15), (4, 14), (5, 12), (6, 11)]]
[0]
2
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=(((((((....)).)))))...............

[[(3, 15), (4, 14), (5, 12), (6, 11)], [(17, 28), (18, 27)]]
[0, 0]
3
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=...((((....)).)).((........)).....

[[(0, 18), (1, 17), (2, 16), (3, 15), (4, 14)], [(7, 32), (8, 31), (9, 30), (10, 29), (11, 28), (12, 27)]]
Pseudok

In [10]:
# make sure to pick the index where the current val is 0 to do GA
model.gradient_ascent(seq=seq, y_prev=y[-1, :, :, 0], idx1=7, idx2=10)

Tensor("input_org:0", shape=(?, ?, 4), dtype=float32)
Tensor("ar_label/Sigmoid:0", shape=(?, ?, ?, 1), dtype=float32)
Tensor("div:0", shape=(?, ?, 4), dtype=float32)
0


AbortedError: Operation received an exception:Status: 5, message: could not create a view primitive descriptor, in file tensorflow/core/kernels/mkl_slice_op.cc:433
	 [[{{node gradients/concatenate_3/concat_grad/Slice}}]]

In [11]:
from utils import DataEncoder

In [12]:
import keras.backend as kb

In [14]:
y_prev=y[-1, :, :, 0]
idx1=7
idx2=10


x_single = DataEncoder.encode_seqs([seq])
if len(y_prev.shape) == 2:
    y_single = y_prev[np.newaxis, :, :, np.newaxis]
elif len(y_prev.shape) == 4:
    y_single = y_prev.copy()
else:
    raise ValueError

# z_repr, _ = self.model_repr.predict([x_single, y_single])
# y = self.model_ar.predict([z_repr, y_single])
# # compute gradient
# # layer_output = self.model_ar.layers[-1].output
# layer_output = self.model_ar.layers[-1].get_output_at(-1)
# loss = kb.mean(layer_output[:, idx1, idx2, :])
# input_node = self.model_ar.layers[0].input[0]  # seq input, not y_prev

# for this part, we assume the model has named layers
y, fe = model._model.predict([x_single, y_single])
# input_node = next(l for l in self._model.layers if l.name == 'input_org').input
# layer_output = next(l for l in self._model.layers if l.name == 'ar_label').get_output_at(-1)
input_node = model._model.input[0]
input_node2 = model._model.input[1]
layer_output = model._model.output[0]
print input_node
print layer_output
loss = kb.mean(layer_output[:, idx1, idx2, :])
# 'input_org'
# 'target_prev'
# 'ar_label'
# 'fe'

# grads = kb.gradients(loss, input_node)[0]
grads = kb.gradients(loss, model._model.input)[0]
grads /= (kb.sqrt(kb.mean(kb.square(grads))) + 1e-5)
# iterate = kb.function([input_node, input_node2], [loss, grads])
# iterate = kb.function([input_node], [loss, grads])
iterate = kb.function(model._model.input, [loss, grads])
print grads
print iterate

x_new = x_single.astype(np.float32)
# add a little bit noise
x_new += 1e-2
# normalize
x_new = x_new / np.sum(x_new, axis=-1)[:, :, np.newaxis]
# run gradient ascent
x_ite = []
for i in range(200):
    print i
    loss_value, grads_value = iterate([x_new, y_single])
#     loss_value, grads_value = iterate([x_new])
    x_new += grads_value * 0.01
    # re-normalize
    x_new = x_new / np.sum(x_new, axis=-1)[:, :, np.newaxis]
    #     print x1_new

    x_ite.append(x_new)


Tensor("input_org:0", shape=(?, ?, 4), dtype=float32)
Tensor("ar_label/Sigmoid:0", shape=(?, ?, ?, 1), dtype=float32)
Tensor("div_2:0", shape=(?, ?, 4), dtype=float32)
0


AbortedError: Operation received an exception:Status: 5, message: could not create a view primitive descriptor, in file tensorflow/core/kernels/mkl_slice_op.cc:433
	 [[{{node gradients_2/concatenate_3/concat_grad/Slice}}]]

In [15]:
print x_single.shape
print y_single.shape
print x_new.shape

(1, 34, 4)
(1, 34, 34, 1)
(1, 34, 4)


In [33]:
# iterate([x_single, np.zeros((1, 34, 34, 1))])
iterate([x_single.astype(np.float16), np.zeros((1, 34, 34, 1)).astype(np.float16)])

AbortedError: Operation received an exception:Status: 5, message: could not create a view primitive descriptor, in file tensorflow/core/kernels/mkl_slice_op.cc:433
	 [[{{node gradients_2/concatenate_3/concat_grad/Slice}}]]

In [25]:
y[0, :, :, 0]

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.1920929e-07,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [2.8610229e-06, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [4.3064356e-05, 7.4416399e-05, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.4762198e-01, 3.9994717e-05, 1.1920929e-07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [27]:
DataEncoder.y_init(34)

array([[[[-1.],
         [ 0.],
         [ 0.],
         ...,
         [ 0.],
         [ 0.],
         [ 0.]],

        [[-1.],
         [-1.],
         [ 0.],
         ...,
         [ 0.],
         [ 0.],
         [ 0.]],

        [[-1.],
         [-1.],
         [-1.],
         ...,
         [ 0.],
         [ 0.],
         [ 0.]],

        ...,

        [[-1.],
         [-1.],
         [-1.],
         ...,
         [-1.],
         [ 0.],
         [ 0.]],

        [[-1.],
         [-1.],
         [-1.],
         ...,
         [-1.],
         [-1.],
         [ 0.]],

        [[-1.],
         [-1.],
         [-1.],
         ...,
         [-1.],
         [-1.],
         [-1.]]]])